In [ ]:
# PSUEDO CODE

#

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import networkx as nx

# Example edge attribute data
edges = pd.DataFrame({
    'edge_id': [1, 2, 3, 4],
    'u': [0, 1, 2, 3],
    'v': [1, 2, 3, 4],
    'sidewalk_quality': [0.8, 0.6, 0.9, 0.7],
    'is_shaded': [1, 0, 1, 1],
    'traffic_density': [0.2, 0.4, 0.1, 0.3]
})

# Observed paths (example)
observed_paths = [
    [0, 1, 2],
    [1, 2, 3],
    [2, 3, 4]
]

# Features and initial weights
X = edges[['sidewalk_quality', 'is_shaded', 'traffic_density']].values
initial_beta = np.zeros(X.shape[1] + 1)  # Including intercept

# Graph construction
G = nx.Graph()
for i, row in edges.iterrows():
    G.add_edge(row['u'], row['v'], edge_id=row['edge_id'], attr=row[['sidewalk_quality', 'is_shaded', 'traffic_density']])

def edge_weight_function(edge_attrs, beta):
    print(edge_attrs)
    return np.exp(beta[0] + np.dot(edge_attrs, beta[1:]))

def compute_path_weight(G, path, beta):
    return sum(edge_weight_function(G[u][v]['attr'], beta) for u, v in zip(path[:-1], path[1:]))

def loss_function(beta, G, observed_paths):
    loss = 0
    for path in observed_paths:
        predicted_path = nx.shortest_path(G, source=path[0], target=path[-1], weight=lambda u, v, data: edge_weight_function(data['attr'], beta))
        predicted_path = set([int(x) for x in predicted_path])
        path = set(path)
        print(path)
        #find the coverage 
        overlap = len(path - predicted_path)
        #check the overlap
        if overlap <= 0: #replace with the overlap threshold
            loss += 1
        print(loss)
        #predicted_path_weight = compute_path_weight(G, predicted_path, beta)
        #observed_path_weight = compute_path_weight(G, path, beta)
        #loss += np.abs(predicted_path_weight - observed_path_weight)  # Sum of absolute differences
    return loss

# Optimization
result = minimize(loss_function, initial_beta, args=(G, observed_paths), method='BFGS')
optimal_beta = result.x

print("Optimal coefficients:", optimal_beta)

# Apply the learned weights to the graph edges
for u, v, data in G.edges(data=True):
    data['weight'] = edge_weight_function(data['attr'], optimal_beta)

# Calculate shortest paths using the learned weights
source, target = 0, 4
shortest_path = nx.shortest_path(G, source=source, target=target, weight='weight')
print("Shortest path:", shortest_path)


In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from scipy.optimize import minimize
import random

# Seed for reproducibility
np.random.seed(42)
random.seed(42)

# Create a graph with at least 500 edges
num_nodes = 200
num_edges = 500

G = nx.gnm_random_graph(num_nodes, num_edges)

# Extract edges from the graph
edges = pd.DataFrame(list(G.edges), columns=['u', 'v'])

# Assign random attributes to each edge
edges['sidewalk_quality'] = np.random.uniform(0, 1, len(edges))
edges['is_shaded'] = np.random.randint(0, 2, len(edges))
edges['traffic_density'] = np.random.uniform(0, 1, len(edges))

# Function to assign attributes from the DataFrame to the NetworkX graph
def assign_attributes_to_graph(G, edges):
    for row in edges.itertuples():
        G.edges[row.u, row.v]['attr'] = pd.Series(
            index=['sidewalk_quality','is_shaded','traffic_density'],
            data=[row.sidewalk_quality,row.is_shaded,row.traffic_density]
        )
        
# Assign the attributes to the graph
assign_attributes_to_graph(G, edges)
edges

,u,v,sidewalk_quality,is_shaded,traffic_density
0,0,184,0.374540,1,0.116898
1,0,153,0.950714,0,0.939832
2,0,99,0.731994,0,0.627708
3,0,85,0.598658,0,0.334906
4,0,111,0.156019,0,0.139272
...,...,...,...,...,...
495,178,197,0.353352,0,0.626220
496,183,191,0.583656,1,0.131245
497,186,193,0.077735,1,0.032526
498,188,189,0.974395,0,0.920848


In [2]:
# Function to generate random paths
def generate_random_paths(G, num_paths):
    paths = []
    nodes = list(G.nodes())
    for _ in range(num_paths):
        source = random.choice(nodes)
        target = random.choice(nodes)
        if nx.has_path(G, source, target) & (source != target):
            path = nx.shortest_path(G, source=source, target=target)
            paths.append(path)
    return paths

# Generate 100 random paths
observed_paths = generate_random_paths(G, 100)
observed_paths[80]

[169, 80, 102]

In [3]:
# Features and initial weights
X = edges[['sidewalk_quality', 'is_shaded', 'traffic_density']].values
initial_beta = np.zeros(X.shape[1] + 1)  # Including intercept
initial_beta


array([0., 0., 0., 0.])

In [4]:
def edge_weight_function(edge_attrs, beta):
    """Calculate the weight of an edge using the exponential function to ensure positivity."""
    return np.exp(beta[0] + np.dot(edge_attrs, beta[1:]))

def compute_path_edges(path):
    """Convert a path to a set of edges."""
    return set(zip(path[:-1], path[1:]))

In [12]:
def loss_function(beta, G, observed_paths):
    """Loss function that measures the discrepancy between observed paths and predicted shortest paths."""
    loss = 0
    for path in observed_paths:
        predicted_path = nx.shortest_path(G, source=path[0], target=path[-1], weight=lambda u, v, data: edge_weight_function(data['attr'], beta))
        predicted_path = set([int(x) for x in predicted_path])
        path = set(path)
        #find the coverage 
        overlap = len(path - predicted_path)
        #check the overlap
        if overlap <= 0: #replace with the overlap threshold
            loss += 1
    return loss

# Optimization to find the optimal beta coefficients
result = minimize(loss_function, initial_beta, args=(G, observed_paths), method='BFGS')
optimal_beta = result.x

print(f"Optimal coefficients: {optimal_beta.round(2)}")

Optimal coefficients: [0. 0. 0. 0.]


In [14]:
# Apply the learned weights to the graph edges
for u, v, data in G.edges(data=True):
    data['weight'] = edge_weight_function(data['attr'], optimal_beta)

# Calculate and print the shortest path using the learned weights for a sample pair
source, target = 0, num_nodes - 1
if nx.has_path(G, source, target):
    shortest_path = nx.shortest_path(G, source=source, target=target, weight='weight')
    print("Shortest path from node 0 to node", num_nodes - 1, ":", shortest_path)
else:
    print(f"No path between node {source} and node {target}")


Shortest path from node 0 to node 199 : [0, 184, 29, 199]
